# Mandatory Assignment 3


## US Jobs and Import Competition from China 
Af: Mathias Nikolaisen(dnt357), Oliver Mattthews(jvc975) & Clara Maj Folkmann(djz279)

### Problem 1

In [40]:
import pandas as pd


data_path = 'A3_X2017S_MetricsI.dta'
df = pd.read_stata(data_path)

# Beregn beskrivende statistik for hver periode
summary_stats_period_0 = df[df['t2'] == 0][['dsL', 'dIPWusch', 'college', 'routine', 'dIPWusmx', 'foreignborn', 'dIPWotch', 'dIPWukch', 'dIPWotmx']].describe()
summary_stats_period_1 = df[df['t2'] == 1][['dsL', 'dIPWusch', 'college', 'routine', 'dIPWusmx', 'foreignborn', 'dIPWotch', 'dIPWukch', 'dIPWotmx']].describe()

# Udskriv summariske statistikker for begge perioder
print("1990-2000 (t2 = 0):")
print(summary_stats_period_0)

print("2000-2007 (t2 = 1):")
print(summary_stats_period_1)

1990-2000 (t2 = 0):
              dsL    dIPWusch     college  ...    dIPWotch    dIPWukch    dIPWotmx
count  662.000000  662.000000  662.000000  ...  662.000000  662.000000  662.000000
mean    -0.957021    1.186600   42.097832  ...    1.016592    0.728172    0.014953
std      2.240247    1.838066    8.530094  ...    1.083533    1.053982    0.112670
min    -19.167072   -0.075351   19.943975  ...   -0.467243   -0.687543   -1.519587
25%     -2.084500    0.248525   35.936215  ...    0.282036    0.122727    0.000772
50%     -0.685493    0.744347   42.515060  ...    0.663031    0.544912    0.012937
75%      0.379791    1.426852   47.567284  ...    1.398126    1.002490    0.037991
max      5.664484   25.405270   62.898552  ...    8.616317   13.057814    0.516513

[8 rows x 9 columns]
2000-2007 (t2 = 1):
              dsL    dIPWusch     college  ...    dIPWotch    dIPWukch    dIPWotmx
count  662.000000  662.000000  662.000000  ...  662.000000  662.000000  662.000000
mean    -2.338758    2.67

In [41]:
import statsmodels.api as sm

y = df['dsL']
X = df[['t2', 'dIPWusch', 'college', 'foreignborn', 'routine']]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    dsL   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     159.9
Date:                Wed, 23 Oct 2024   Prob (F-statistic):          6.08e-133
Time:                        12:03:27   Log-Likelihood:                -2788.1
No. Observations:                1324   AIC:                             5588.
Df Residuals:                    1318   BIC:                             5619.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           5.0983      0.544      9.371      

### Problem 2

In [42]:
from linearmodels.iv import IV2SLS

y = df['dsL']

X = df[['t2', 'dIPWusch', 'college', 'foreignborn', 'routine']]
Z = df['dIPWotch']
endog = df['dIPWotch']

X = sm.add_constant(X)

iv_model = IV2SLS(y, X, endog=endog, instruments=Z).fit()

print(iv_model.summary)


                          IV-2SLS Estimation Summary                          
Dep. Variable:                    dsL   R-squared:                      0.4058
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4030
No. Observations:                1324   F-statistic:                    618.48
Date:                Wed, Oct 23 2024   P-value (F-stat)                0.0000
Time:                        12:03:27   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
const           4.9502     0.4773     10.371     0.0000      4.0147      5.8857
t2             -1.1932     0.1438    -8.2955    

In [43]:
y = df['dsL']
X = df[['t2', 'college', 'foreignborn', 'routine']]
endog = df['dIPWusch']
Z = df[['dIPWotch', 'dIPWukch']]  

X = sm.add_constant(X)

iv_model = IV2SLS(y, X, endog=endog, instruments=Z).fit()
iv_resid = iv_model.resids
help_reg = sm.OLS(iv_resid, sm.add_constant(Z)).fit()
nR2 = help_reg.nobs * help_reg.rsquared

from scipy.stats import chi2
p_value = chi2.sf(nR2, df=Z.shape[1] - 1)

print(iv_model.summary)

print(f"Hansen's J-statistic: {nR2}")
print(f"P-value for J-statistic: {p_value}")


                          IV-2SLS Estimation Summary                          
Dep. Variable:                    dsL   R-squared:                      0.3426
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3401
No. Observations:                1324   F-statistic:                    584.90
Date:                Wed, Oct 23 2024   P-value (F-stat)                0.0000
Time:                        12:03:27   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
const           4.6095     0.5120     9.0030     0.0000      3.6060      5.6129
t2             -1.1333     0.1743    -6.5015    

In [44]:
y = df['dsL']  

X = df[['t2', 'college', 'foreignborn', 'routine']] 

endog = df[['dIPWusch', 'dIPWusmx']] 

Z = df[['dIPWotch', 'dIPWotmx']] 

X = sm.add_constant(X)

iv_model = IV2SLS(y, X, endog=endog, instruments=Z).fit()

print(iv_model.summary.as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &        dsL         & \textbf{  R-squared:         } &      0.3088      \\
\textbf{Estimator:}        &      IV-2SLS       & \textbf{  Adj. R-squared:    } &      0.3057      \\
\textbf{No. Observations:} &        1324        & \textbf{  F-statistic:       } &      538.01      \\
\textbf{Date:}             &  Wed, Oct 23 2024  & \textbf{  P-value (F-stat)   } &      0.0000      \\
\textbf{Time:}             &      12:03:28      & \textbf{  Distribution:      } &     chi2(6)      \\
\textbf{Cov. Estimator:}   &       robust       & \textbf{                     } &                  \\
\textbf{}                  &                    & \textbf{                     } &                  \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                     & \textbf{Parameter} & \textbf{Std. Err.} & \textbf{T-stat} & \textbf{P-value} & \textbf{Lower CI} & \textbf{Upper CI}  \\
\midrule
\textbf{const}       &       

### Problem 3

## Problem 4

## Problem 5

In [ ]:
import numpy as np
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
import matplotlib.pyplot as plt

n = 1000
replications = 500

beta_0 = 4
beta_1 = 3
theta = 1
rho_values = [-0.5, 0, 0.5, 1] 

ols_estimates = []
iv_estimates = []

np.random.seed(1234)

for rho in rho_values:
    ols_estimates_rho = []
    iv_estimates_rho = []
    
    for _ in range(replications):

        x_star = np.random.normal(1, np.sqrt(4), n) 
        u = np.random.normal(0, 1, n) 
        epsilon = np.random.normal(0, 1, n) 
        mu = np.random.normal(0, 1, n)  
        
        x = x_star + epsilon  
        eta = rho * epsilon + mu  
        z = theta * x_star + eta  
        

        y = beta_0 + beta_1 * x_star + u 

        X_ols = sm.add_constant(x) 
        ols_model = sm.OLS(y, X_ols).fit()
        ols_estimates_rho.append(ols_model.params[1])
        
        X_iv = sm.add_constant(np.ones_like(x))  
        iv_model = IV2SLS(y, X_iv, endog=x, instruments=z).fit()
        iv_estimates_rho.append(iv_model.params[1])
    
    ols_estimates.append(ols_estimates_rho)
    iv_estimates.append(iv_estimates_rho)

    plt.figure(figsize=(12, 6))
    plt.hist(ols_estimates_rho, bins=30, alpha=0.5, label="OLS", color="blue")
    plt.hist(iv_estimates_rho, bins=30, alpha=0.5, label="IV", color="red")
    plt.title(f"Distribution of OLS and IV Estimates (rho = {rho})")
    plt.xlabel("Estimat")
    plt.ylabel("Frekvens")
    plt.legend()
    plt.show()